### Experiment:  Comparison of RAG models using RAGAS framework

**Background:**  Generate RAGAS based measurement for proposed RAG API candidates.

**Test Approach:** Test RAG Parameters:

* RAG_0

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()


True

In [3]:
# Common import
from deh.assessment import QASetRetriever
from deh import settings
from deh.eval import generate_experiment_dataset

import pandas as pd
import json
import os
from pathlib import Path

# For RAGAS evaluation
from datasets import Dataset
from ragas import evaluate
import ragas.metrics as metrics
from ragas.run_config import RunConfig

/home/ubuntu/miniconda3/envs/deh_measurement/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Test Configuration

In [4]:
num_samples:int = 100
experiment_folder:str = "../../data/evaluation/rag_api_ragas_metrics/"
qa_data_set_file:str = "../../data/qas/squad_qas.tsv"

# Create experiment folder:
if not os.path.exists(experiment_folder):
    Path(experiment_folder).mkdir(parents=True, exist_ok=True)

#### Evaluation Model Setup

In [5]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.llms import LangchainLLMWrapper

# Either local (Ollama) or remote (OpenAI) evaluation models can be used:

# llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4"))
# embedding = OpenAIEmbeddings()


In [6]:
embedding = OllamaEmbeddings(
    base_url=settings.OLLAMA_URL,
    model=settings.ASSESSMENT_EMBEDDING_MODEL,
)

llm = LangchainLLMWrapper(Ollama(
    base_url=settings.OLLAMA_URL,
    model=settings.ASSESSMENT_LLM_MODEL
))

/tmp/ipykernel_162808/277639191.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(
/tmp/ipykernel_162808/277639191.py:6: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = LangchainLLMWrapper(Ollama(


#### Sample QA dataset

In [7]:
qa_set = QASetRetriever.get_qasets(
    file_path = qa_data_set_file,
    sample_size= num_samples
)

print(f"{len(qa_set)} questions sampled from QA corpus ({qa_data_set_file})")

100 questions sampled from QA corpus (../../data/qas/squad_qas.tsv)


### NO_RAG Experimentation

#### Response Generation

In [8]:
def api_endpoint(**kwargs) -> str:
    """Endpoint for context retrieval."""
    hyde= False
    kwargs["h"] = False
    kwargs["e"] = False
    kwargs["k"] = 1
    kwargs["t"] = 0.5
    kwargs["m"] = "llama3.1:8b-instruct-q3_K_L"
    kwargs["lp"] = 2    # no context, 10 words

    query_params = "&".join([f"{key}={kwargs[key]}" for key in kwargs])
    print (query_params)
    return f"http://{settings.API_ANSWER_ENDPOINT}/answer?{query_params}"

def convert(response) -> pd.DataFrame:
    """Converts retrieved JSON response to Pandas DataFrame"""
    response_df = pd.json_normalize(
        data=response["response"], record_path="context", meta=["answer","question", "hyde", ["evaluation", "grade"]]
    )

    # Add reference/evaluation values:
    response_df["reference.ground_truth"] = response["reference"]["ground_truth"]
    response_df["reference.is_impossible"] = response["reference"]["is_impossible"]

    # Add full JSON response incase needed:
    response_df["json"] = json.dumps(response)
    return response_df

exp_df = generate_experiment_dataset(qa_set, convert, api_endpoint)

# Store the generated response:
exp_df.to_pickle( f"{experiment_folder}/no_rag_b1.pkl" )
exp_df[0:1]


Processing 1 of 100 question/answer pairs.
q=What%20is%20the%20power-to-weight%20ratio%20of%20a%20steam%20plant%20compared%20to%20that%20of%20an%20internal%20combustion%20engine%3F&h=False&e=False&k=1&t=0.5&m=llama3.1:8b-instruct-q3_K_L&lp=2
Processing 2 of 100 question/answer pairs.
q=Under%20which%20leader%20did%20the%20Huguenots%20fight%20in%20this%20conflict%3F&h=False&e=False&k=1&t=0.5&m=llama3.1:8b-instruct-q3_K_L&lp=2
Processing 3 of 100 question/answer pairs.
q=When%20did%20this%20attempt%20take%20place%3F&h=False&e=False&k=1&t=0.5&m=llama3.1:8b-instruct-q3_K_L&lp=2
Processing 4 of 100 question/answer pairs.
q=WHy%20was%20the%20Merit%20network%20formed%20in%20Michigan%20&h=False&e=False&k=1&t=0.5&m=llama3.1:8b-instruct-q3_K_L&lp=2
Processing 5 of 100 question/answer pairs.
q=Does%20the%20residential%20architecture%20of%20the%20Tower%20District%20compare%20or%20contrast%20with%20other%20part%20of%20Fresno%3F&h=False&e=False&k=1&t=0.5&m=llama3.1:8b-instruct-q3_K_L&lp=2
Processing

,id,page_content,type,metadata.source,metadata.similarity_score,answer,question,hyde,evaluation.grade,reference.ground_truth,reference.is_impossible,json,reference_id
0,None,The weight of boilers and condensers generally...,Document,/data/contexts/context_1078.context,0.358998,Internal combustion engines have higher power-...,What is the power-to-weight ratio of a steam p...,False,,lower,False,"{""response"": {""question"": ""What is the power-t...",1


##### RAGAS Evaluation Responses

In [10]:
# Convert to Dataset
responses_df = pd.read_pickle(f"{experiment_folder}/no_rag_b1.pkl")

responses_df = responses_df.groupby("reference_id").agg(
    retrieved_contexts = ('page_content', lambda x: list(x)),
    question = ('question','first'),
    ground_truth = ('reference.ground_truth', 'first'),
    answer = ('answer', 'first')
    )

responses_df[0:1]


,retrieved_contexts,question,ground_truth,answer
reference_id,,,,
1,[The weight of boilers and condensers generall...,What is the power-to-weight ratio of a steam p...,lower,Internal combustion engines have higher power-...


In [ ]:
# Single iteration for Exception management

from ragas import evaluate
from ragas.metrics import Faithfulness, FactualCorrectness

results_df = []

for i in range(len(responses_df)):
   print (f"Processing {i} of {len(responses_df)}")
   df=responses_df.iloc[i:i+1]
   ds = Dataset.from_pandas( df)
   
   metrics = [FactualCorrectness(), Faithfulness()]
   try:
      evaluation_ds = evaluate(
                     dataset=ds, 
                     metrics=metrics, 
                     llm=llm,
                     run_config=RunConfig(
                        max_workers=1,
                        max_retries=1
                     ))
   
      eval_df = evaluation_ds.to_pandas()
      results_df.append(eval_df)
   except:
      pass

appended_ds = pd.concat(results_df)
appended_ds.to_pickle( f"{experiment_folder}/no_rag_b1-results.pkl" )

appended_ds[0:5]

Processing 0 of 100


Evaluating: 100%|██████████| 2/2 [00:19<00:00,  9.71s/it]


Processing 1 of 100


Evaluating: 100%|██████████| 2/2 [00:17<00:00,  8.76s/it]


Processing 2 of 100


...

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt claim_decomposition_prompt failed to parse output: The output parser failed to parse the output including retries.
Exception raised in Job[0]: RagasOutputParserException(The output parser failed to parse the output including retries.)
Evaluating: 100%|██████████| 2/2 [00:19<00:00,  9.97s/it]


Processing 23 of 100


Evaluating: 100%|██████████| 2/2 [00:29<00:00, 14.63s/it]


Processing 24 of 100


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

### RAG_FINAL Experimentation

In [9]:
def api_endpoint(**kwargs) -> str:
    """Endpoint for context retrieval."""
    hyde= False
    kwargs["h"] = False
    kwargs["e"] = False
    kwargs["k"] = 2
    kwargs["t"] = 0.5
    kwargs["m"] = "qwen2.5:7b"
    kwargs["lp"] = 0    # no context, 10 words

    query_params = "&".join([f"{key}={kwargs[key]}" for key in kwargs])
    print (query_params)
    return f"http://{settings.API_ANSWER_ENDPOINT}/answer?{query_params}"

def convert(response) -> pd.DataFrame:
    """Converts retrieved JSON response to Pandas DataFrame"""
    response_df = pd.json_normalize(
        data=response["response"], record_path="context", meta=["answer","question", "hyde", ["evaluation", "grade"]]
    )

    # Add reference/evaluation values:
    response_df["reference.ground_truth"] = response["reference"]["ground_truth"]
    response_df["reference.is_impossible"] = response["reference"]["is_impossible"]

    # Add full JSON response incase needed:
    response_df["json"] = json.dumps(response)
    return response_df

exp_df = generate_experiment_dataset(qa_set, convert, api_endpoint)

# Store the generated response:
exp_df.to_pickle( f"{experiment_folder}/rag_final_b1.pkl" )
exp_df[0:1]


Processing 1 of 100 question/answer pairs.
q=What%20is%20the%20power-to-weight%20ratio%20of%20a%20steam%20plant%20compared%20to%20that%20of%20an%20internal%20combustion%20engine%3F&h=False&e=False&k=2&t=0.5&m=qwen2.5:7b&lp=0
Processing 2 of 100 question/answer pairs.
q=Under%20which%20leader%20did%20the%20Huguenots%20fight%20in%20this%20conflict%3F&h=False&e=False&k=2&t=0.5&m=qwen2.5:7b&lp=0
Processing 3 of 100 question/answer pairs.
q=When%20did%20this%20attempt%20take%20place%3F&h=False&e=False&k=2&t=0.5&m=qwen2.5:7b&lp=0
Processing 4 of 100 question/answer pairs.
q=WHy%20was%20the%20Merit%20network%20formed%20in%20Michigan%20&h=False&e=False&k=2&t=0.5&m=qwen2.5:7b&lp=0
Processing 5 of 100 question/answer pairs.
q=Does%20the%20residential%20architecture%20of%20the%20Tower%20District%20compare%20or%20contrast%20with%20other%20part%20of%20Fresno%3F&h=False&e=False&k=2&t=0.5&m=qwen2.5:7b&lp=0
Processing 6 of 100 question/answer pairs.
q=How%20do%20you%20pronounce%20Fresno%3F&h=False&e=F

,id,page_content,type,metadata.source,metadata.similarity_score,answer,question,hyde,evaluation.grade,reference.ground_truth,reference.is_impossible,json,reference_id
0,None,The weight of boilers and condensers generally...,Document,/data/contexts/context_1078.context,0.358998,Lower due to boiler and condenser weight const...,What is the power-to-weight ratio of a steam p...,False,,lower,False,"{""response"": {""question"": ""What is the power-t...",1


In [ ]:
# Convert to Dataset
responses_df = pd.read_pickle(f"{experiment_folder}/rag_final_b1.pkl")

responses_df = responses_df.groupby("reference_id").agg(
    retrieved_contexts = ('page_content', lambda x: list(x)),
    question = ('question','first'),
    ground_truth = ('reference.ground_truth', 'first'),
    answer = ('answer', 'first')
    )

responses_df[0:1]

In [ ]:
# Single iteration for Exception management

from ragas import evaluate
from ragas.metrics import Faithfulness, FactualCorrectness

results_df = []

for i in range(len(responses_df)):
   print (f"Processing {i} of {len(responses_df)}")
   df=responses_df.iloc[i:i+1]
   ds = Dataset.from_pandas( df)
   
   metrics = [FactualCorrectness(), Faithfulness()]
   try:
      evaluation_ds = evaluate(
                     dataset=ds, 
                     metrics=metrics, 
                     llm=llm,
                     run_config=RunConfig(
                        max_workers=1,
                        max_retries=1
                     ))
   
      eval_df = evaluation_ds.to_pandas()
      results_df.append(eval_df)
   except:
      pass

appended_ds = pd.concat(results_df)
appended_ds.to_pickle( f"{experiment_folder}/rag_final_b1-results.pkl" )
appended_ds[0:5]

#### View Results

In [23]:
no_rag_results_df = pd.read_pickle(f"{experiment_folder}/no_rag_b1-results.pkl")
no_rag_results_df[0:5]
no_rag_results_df.to_csv(f"{experiment_folder}/no_rag_b1-results.csv")


In [25]:
rag_final_results_df = pd.read_pickle(f"{experiment_folder}/rag_final_b1-results.pkl")
rag_final_results_df[0:5]
rag_final_results_df.to_csv(f"{experiment_folder}/rag_final_b1-results.csv")


In [21]:
# NO_RAG:
print( f"""
Count: {len(no_rag_results_df)}
Factual_Correctness: {no_rag_results_df["factual_correctness"].mean()}
Faithfulness: {no_rag_results_df["faithfulness"].mean()}
""")


Count: 51
Factual_Correctness: 0.42058823529411754
Faithfulness: 0.4852941176470588



In [22]:
# RAG_FINAL:
print( f"""
Count: {len(rag_final_results_df)}
Factual_Correctness: {rag_final_results_df["factual_correctness"].mean()}
Faithfulness: {rag_final_results_df["faithfulness"].mean()}
""")


Count: 20
Factual_Correctness: 0.48250000000000004
Faithfulness: 0.7

